# Goal

The aim of this notebook is to provide an example of how to execute `Struo` to build a custom database. Commands with an **execute from notebook** comment are for reference, only those saying **execute from terminal** are needed to install the software and build the databases

# Var 

In [2]:
# Execute from notebook
import os
import pandas as pd

/ebio/abt3_projects/software/miniconda3_gt4.4/envs/py3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# Download `Struo`

First, we need to move to the folder where we're going to download the repository and create the databases. 

```bash
# Execute from terminal
cd /ebio/abt3_projects/Struo/example_build
```

**Note**: be sure to change this path to the path in your local system.

Next, we'll create folders for other needed files and an output directory

```bash
# Execute from terminal
mkdir Files
mkdir Genomes
```

Next, we'll clone the repo

```bash
# Execute from terminal
git clone git@github.com:leylabmpi/Struo.git 
```

# Set `conda` environment

`Struo` needs a working conda installation. The required packages and their exact versions are listed in the README and in the `struo_env.yaml` file. This file can be used to create a new conda environment called `struo` with the needed software and dependencies. Make sure that you have a working conda installation and execute: 

```bash
# Execute from terminal
conda env create --file=./Struo/tutorial/struo_env.yaml
```

Additionally, you will need a working installation of `HUMANn2` to download required files, although it doesn't have to be part of this `conda` environment. Note that `Struo` will install all other software for you, so there is no need to include it in the `conda` envinronment

# Download UniRef databases for `HUMANn2`

We will need a UniRef diamond database for the `HUMANn2` database construction (e.g., UniRef50). See the "Download a translated search database" section of the [HUMAN2 docs](https://bitbucket.org/biobakery/humann/wiki/Home#markdown-header-download-a-translated-search-database).

To download the UniRef50 database using `HUMAN2`, use the following command:
```bash
# Execute from terminal
humann2_databases --download uniref uniref50_diamond ./Files
```

# Download genomes

The software and files are almost ready. We can now prepare the tables with the information of the genomes that we wish to include. For this tutorial, we'll be using a very small subset of the GTDB genomes. The `GTDB_metadata_filter.R` script downloads the metadata tables provided by the GTDB and filters them based on the completeness and contamination thresholds we set as default; if you wish to set different cut-off values, modify this script. For release 86 of the GTDB, these files can be found [here](https://data.ace.uq.edu.au/public/gtdb/data/releases/release86/86.0/).

To download the *Archaea* genomes from GTDB using the default filtering parameters, we use

```bash
# Execute from terminal
cd Struo
conda activate struo

Rscript util_scripts/GTDB_metadata_filter.R https://data.ace.uq.edu.au/public/gtdb/data/releases/release86/86.0/arc_metadata_r86.tsv --output ../Files/Archaea_genomes.txt
```

The resulting file will be used for the rest of the pipeline. The fields are described in the `README.md` file. If you want to include other genomes you downloaded, you can add a column called  `fasta_file_path` with the location of these genomes

In [9]:
# Execute from notebook
pd.read_csv('/ebio/abt3_projects/Struo/example_build/Files/Archaea_genomes.txt', sep='\t', header=0).head()

,ncbi_organism_name,ncbi_genbank_assembly_accession,scaffold_count,contig_count,gc_percentage,genome_size,checkm_completeness,checkm_contamination,checkm_strain_heterogeneity,ncbi_assembly_level,ncbi_refseq_category,ncbi_species_taxid,ncbi_taxonomy,gtdb_taxonomy,mimag_high_quality,gtdb_representative
0,Thermococcus litoralis DSM 5473,GCA_000246985.3,1,1,43.088029,2215172,99.50,0.50,0.0,Complete Genome,representative genome,2265,d__Archaea;p__Euryarchaeota;c__Thermococci;o__...,d__Archaea;p__Euryarchaeota;c__Thermococci;o__...,t,t
1,Methanolobus tindarius DSM 2278,GCA_000504205.1,1,1,39.789852,3151883,99.67,0.00,0.0,Contig,representative genome,2221,d__Archaea;p__Euryarchaeota;c__Methanomicrobia...,d__Archaea;p__Halobacterota;c__Methanosarcinia...,t,t
2,Candidatus Nitrosopelagicus brevis,GCA_000812185.1,1,1,33.157756,1232128,99.51,0.00,0.0,Complete Genome,na,1410606,d__Archaea;p__Thaumarchaeota;c__;o__;f__;g__Ca...,d__Archaea;p__Crenarchaeota;c__Nitrososphaeria...,t,t
3,Methanococcus voltae A3,GCA_000006175.2,1,1,28.594594,1936387,99.05,0.00,0.0,Complete Genome,representative genome,2188,d__Archaea;p__Euryarchaeota;c__Methanococci;o_...,d__Archaea;p__Euryarchaeota;c__Methanococci;o_...,t,t
4,Halorubrum hochstenium ATCC 700873,GCA_000337075.1,64,64,69.130537,3037532,99.38,0.19,0.0,Contig,na,1227480,d__Archaea;p__Euryarchaeota;c__Halobacteria;o_...,d__Archaea;p__Halobacterota;c__Halobacteria;o_...,f,t


For the sake of speed, let's create a database with just the first 10 genomes in the `Archaea_genomes.txt` file

```bash
# Execute from terminal
head ../Files/Archaea_genomes.txt > ../Files/test_genomes.txt
```

We can now download these genomes. Note that the `genome_download.R` script adds the path to the fasta file of each of the downloaded genomes and prints it to the standard out

```bash 
# Execute from terminal
Rscript util_scripts/genome_download.R --output ../Genomes ../Files/test_genomes.txt > ../Files/Samples_file.txt
```

# Edit the configuration file

We need to modify the `config.yaml` file to specify the parameters of the pipeline, including the input, output and temporary directories. *Remember* to modify the paths to the input, output diamond databases and temporary directories. In addition, for this test, the file has other parameters modified: the `diamond_db_to_mem` parameter is set to **False**



It should look something like this:

In [3]:
# Execute from notebook
!cat /ebio/abt3_projects/Struo/example_build/Struo/config.yaml

#-- I/O --#
# file listing samples and associated data
samples_file: ../Files/Samples_file.txt

## column names in samples table
samples_col: 'ncbi_organism_name'
fasta_file_path_col: 'fasta_file_path'
taxID_col: 'ncbi_species_taxid'    
taxonomy_col: 'ncbi_taxonomy'   

# output location
output_dir: ../Output

# temporary file directory (your username will be added automatically)
tmp_dir: ../tmp/

#-- databases to create --#
# Replace "Create" with "Skip" to skip creation of any of these
# Note that braken relies on the kraken2 database
databases:
  kraken2: Create
  bracken: Create
  humann2_bowtie2: Create
  humann2_diamond: Create

# output database name
db_name: GTDB-custom

#-- keep intermediate files required for re-creating DBs (eg., w/ more genomes) --#
# If "True", the intermediate files are saved to `output_dir`
# Else, the intermediate files are temporarily stored in `temp_folder`
keep_intermediate: True
use_ancient: True

#-- if custom NCBI taxdump files (or just Skip) --#

# (Optional) Build additional `conda` environments

Next, we build a `conda` environment for each of the metagenome profilers. `Struo` takes care of this for you, just run:

```bash
# Execute from terminal
## Make sure you are in the pipeline folder
## Create envs
conda activate struo && snakemake --use-conda --create-envs-only
```

*Note*: this step is optional, if not run, the `conda` environments will be built the first time the pipeline is executed.

We can see the dependencies of each `conda`. Be sure to change the `pipeline_folder` variable to the path in your local system.

In [8]:
# Execute from notebook
pipeline_folder = "/ebio/abt3_projects/Struo/example_build/Struo"
sessionInfo = "find {0} -name '*.yaml' | xargs head -n 1000".format(os.path.join(pipeline_folder, 'bin', 'envs'))
# Command
print(sessionInfo)
!$sessionInfo

find /ebio/abt3_projects/Struo/example_build/Struo/bin/envs -name '*.yaml' | xargs head -n 1000
==> /ebio/abt3_projects/Struo/example_build/Struo/bin/envs/humann2.yaml <==
channels: !!python/tuple
- bioconda
dependencies:
- pigz
- bioconda::vsearch
- bioconda::prodigal
- bioconda::diamond=0.8.36

==> /ebio/abt3_projects/Struo/example_build/Struo/bin/envs/kraken2.yaml <==
channels: !!python/tuple
- bioconda
dependencies:
- libiconv
- bioconda::kraken2
- bioconda::bracken



# Execute the pipeline

Now that the software is installed, the genomes are downloaded and the pipeline is configured, we can build the databases! This process may take a while, so it is a good idea to run it screened. 

**Note** that the number of cores provided will depend on the machine you are using, so change accordingly

```bash
# Execute from terminal
conda activate struo 

screen -L -S struo_test bash -c "snakemake --use-conda --cores 10"
```

# Check the files

The files should be ready. We can see they are in the output directory

In [12]:
#Execute from notebook
!ls -lha /ebio/abt3_projects/Struo/example_build/Databases/humann2/

total 58M
drwxrwxr-x  5 jdelacuesta ebio 4.0K Oct 29 19:21 .
drwxrwxr-x  6 jdelacuesta ebio   86 Oct 29 16:55 ..
-rw-rw-r--  1 jdelacuesta ebio  13M Oct 29 19:21 all_genes_annot.1.bt2
-rw-rw-r--  1 jdelacuesta ebio 4.9M Oct 29 19:21 all_genes_annot.2.bt2
-rw-rw-r--  1 jdelacuesta ebio 204K Oct 29 19:21 all_genes_annot.3.bt2
-rw-rw-r--  1 jdelacuesta ebio 4.9M Oct 29 19:21 all_genes_annot.4.bt2
-rw-rw-r--  1 jdelacuesta ebio 8.5M Oct 29 19:21 all_genes_annot.dmnd
-rw-rw-r--  1 jdelacuesta ebio 4.0M Oct 29 19:21 all_genes_annot.faa.gz
-rw-rw-r--  1 jdelacuesta ebio 5.8M Oct 29 19:21 all_genes_annot.fna.gz
-rw-rw-r--  1 jdelacuesta ebio  13M Oct 29 19:21 all_genes_annot.rev.1.bt2
-rw-rw-r--  1 jdelacuesta ebio 4.9M Oct 29 19:21 all_genes_annot.rev.2.bt2
-rw-rw-r--  1 jdelacuesta ebio    0 Oct 29 19:21 bowtie2_build.done
drwxrwxr-x  2 jdelacuesta ebio 4.0K Oct 29 19:21 nuc_filtered
drwxrwxr-x 11 jdelacuesta ebio 4.0K Oct 29 19:15 prodigal
drwxrwxr-x  2 jdelacuesta ebio 4.0K Oct 29 19:21 pr

In [13]:
#Execute from notebook
!ls -lha /ebio/abt3_projects/Struo/example_build/Databases/kraken2/

total 43M
drwxrwxr-x 5 jdelacuesta ebio  328 Oct 29 19:22 .
drwxrwxr-x 6 jdelacuesta ebio   86 Oct 29 16:55 ..
drwxrwxr-x 2 jdelacuesta ebio 4.0K Oct 29 16:43 added
-rw-rw-r-- 1 jdelacuesta ebio  664 Oct 29 19:22 database100mers.kmer_distrib
-rw-rw-r-- 1 jdelacuesta ebio  19K Oct 29 19:22 database100mers.kraken
-rw-rw-r-- 1 jdelacuesta ebio  654 Oct 29 19:22 database150mers.kmer_distrib
-rw-rw-r-- 1 jdelacuesta ebio  18K Oct 29 19:22 database150mers.kraken
-rw-rw-r-- 1 jdelacuesta ebio 563K Oct 29 19:22 database.kraken
-rw-rw-r-- 1 jdelacuesta ebio  43M Oct 29 19:21 hash.k2d
drwxrwxr-x 3 jdelacuesta ebio   27 Oct 29 16:43 library
-rw-rw-r-- 1 jdelacuesta ebio   56 Oct 29 19:21 opts.k2d
-rw-rw-r-- 1 jdelacuesta ebio  11K Oct 29 19:21 seqid2taxid.map
-rw-rw-r-- 1 jdelacuesta ebio 3.6K Oct 29 19:21 taxo.k2d
drwxrwxr-x 2 jdelacuesta ebio 4.0K Oct 29 19:21 taxonomy


# Optional: add genomes to an existing database

One of the features of the pipeline is that you can expand an already existing database with more genomes. This avoids having to rerun the complete pipeline from scratch. 

**Important**: to do this, you must retain files from the processing of previous genomes so that their are not reprocessed and only the newly added are. This is done by using the `keep_intermediate: True` parameter in the `config.yaml` file, which is the default setting.

We'll need to add the new genomes to the old `test_genomes.txt` file to retrieve the genome sequences from NCBI. Remember that in the first section we used the first ten genomes from the *Archaea* collection of the GTDB, let's use the first 15 (the original 10 plus 5 new genomes)

```bash
# Execute from terminal
head -n 15 ../Files/Archaea_genomes.txt > ../Files/test_add.txt
```

Next, we execute the `genome_download.R` script as above.

```bash
# Execute from terminal
conda activate struo

Rscript util_scripts/genome_download.R --output ../Genomes ../Files/test_add.txt > ../Files/Samples_add.txt
```

We now have the genome sequences and an updated samples file.

Before we can execute the pipeline again, we have to make sure that the following output files from the original execution are removed from the output directory

* humann2 database
  * all_genes_annot.dmnd
* kraken database
  * hash.k2d
  * taxo.k2d
* bracken database
  * database100mers.kraken
  * database150mers.kraken
  
```bash
# Execute from terminal
rm ../Databases/kraken2/database100mers.kraken ../Databases/kraken2/database150mers.kraken ../Databases/kraken2/hash.k2d ../Databases/kraken2/taxo.k2d ../Databases/humann2/all_genes_annot.dmnd
```

**Note** that if you want to keep these files, you can just rename them

Next we can prepare the `config.yaml` file. We just need to specify the name of the new samples file.

Finally, we execute the pipeline just like we did the first time:

```bash
# Execute from terminal
conda activate struo 

screen -L -S struo_test bash -c "snakemake --use-conda --cores 10"
```

# Optional: re-running the pipeline with HUMANn2-formatted genes. 

It is possible to obtain genes not from genome assemblies, but from protein assemblies (e.g., genes assembled from metagenomes using PLASS https://github.com/soedinglab/plass). Gene sequences obtained this way can also be added to HUMANn2 databases using Struo.

The genes should be in nucleotide or amino acid fasta format, and the files should be gzipped. We will use two example files that are located in the `data/tests` folder within the Struo folder. 

In [22]:
# Execute in notebook
!zcat /ebio/abt3_projects/Struo/example_build/Struo/tests/data/clusters_rep-seqs.faa.gz | head
!zcat /ebio/abt3_projects/Struo/example_build/Struo/tests/data/clusters_rep-seqs.fna.gz | head

>UniRef50_U6EEN0|117|g__Candidatus_Methanoperedenaceae.s__Candidatus_Methanoperedens_sp_BLZ1__taxID1719120
PLTTVIRPNLMTKPATLIIPKVTVGDLDDAAKVFGPAQTAVGRAVADAVEEGYIPKDIVEDIVINVSVFIDPSAKNYRKIYQYNYGATKLAIRRAMEGYPSIDKVLAEKDRGTHPIM
>UniRef50_W1I8X6|316|g__Clostridiaceae.s__uncultured_Clostridium_sp___taxID59620
YRLSTLNCGGEDFAVACMRSNLRYEKNQRREDVKSHHYIISFDPRDAVDNGLTVDRAQALGEEFCRKQFPGHQAIVCTHPDGHNHSGNIHVHIVINSLRIEEVPFLPYMDRPADTKAGCKHRCTDAALRYFKSEVMEMCHREGLYQIDLLNGSKNRVTDREYWAQKKGQAALDKQNAPMIAGGITPRQTKFETNKEKLRQTIRAALSAATSFEDFSSLLLREGVAVKESRGRLSYLTPDRTKPITARKLGDDFDRAAVLALLEQNAHRAAEQTATVPEYPRNIRERLQGKKAVQTTPEKDGIQRMVDRAAKRAEGK
>UniRef50_I3YK69|116|g__Rikenellaceae.s__unclassified__taxID679935
MEPANFPEIYTGNEDESDRMQDIAGCFDPIIPRNDGFQYDIEAAASDVCHGKDKWKLPILPVAPGKIIPGRSAVPPSPAPGRKNHPRRGCGIFPPLHLRRGLLTLPEATDPFGI
>UniRef50_R7FWB0|72|g__Peptococcaceae.s__Desulfosporosinus_sp_OL__taxID1888891
FSIPVHQFIFNGGNVDKKFFMDNLQLEESEYALIRDPQRGSCLYKCGNERYLLQVQAPEYKQALFGTAGGR
>UniRef50_K1TC45|153|g__Rikenellaceae.s__unc

We need to specify the path to these files in the `config.yaml` file under the `humann2_nuc_seqs` and `humann2_nuc_seqs` options

In [23]:
# Execute from notebook
!cat /ebio/abt3_projects/Struo/example_build/Struo/config.yaml

#-- I/O --#
# file listing samples and associated data
samples_file: /ebio/abt3_projects/Struo/example_build/Files/Samples_add.txt

## column names in samples table
samples_col: 'ncbi_organism_name'
fasta_file_path_col: 'fasta_file_path'
taxID_col: 'ncbi_species_taxid'    
taxonomy_col: 'ncbi_taxonomy'   

# output location
output_dir: /ebio/abt3_projects/Struo/example_build/Databases

#-- databases to create --#
# Replace "Create" with "Skip" to skip creation of any of these
# Note that braken relies on the kraken2 database
databases:
  kraken2: Create
  bracken: Create
  humann2_bowtie2: Create
  humann2_diamond: Create

#-- keep intermediate files required for re-creating DBs (eg., w/ more genomes) --#
# If "True", the intermediate files are saved to `output_dir`
# Else, the intermediate files are temporarily stored in `temp_folder`
keep_intermediate: True

#-- if custom NCBI taxdump files (or just Skip) --#
names_dmp: Skip 
nodes_dmp: Skip 

#-- software parameters --#
# `vsearch_p

Finally, we execute the pipeline just like we did the first time:

```bash
# Execute from terminal
conda activate struo

screen -L -S struo_test bash -c "snakemake --use-conda --cores 10"
```

# Session Info

The conda environment used to execute the pipeline in this tutorial contains the following packages

In [38]:
# Execute from notebook
!conda list -n struo

# packages in environment at /ebio/abt3_projects/software/miniconda3_gt4.4/envs/struo:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                        main  
_r-mutex                  1.0.1               anacondar_1    conda-forge
aioeasywebdav             2.4.0                 py36_1000    conda-forge
aiohttp                   3.6.1            py36h516909a_0    conda-forge
appdirs                   1.4.3                      py_1    conda-forge
asn1crypto                1.0.1                    py36_0    conda-forge
async-timeout             3.0.1                   py_1000    conda-forge
attrs                     19.2.0                     py_0    conda-forge
bcrypt                    3.1.7            py36h7b6447c_0  
binutils_impl_linux-64    2.31.1               h6176602_1  
binutils_linux-64         2.31.1               h6176602_9    conda-forge
boto3                     1.9.245                    py_0    conda-forge
botocor